<a href="https://colab.research.google.com/github/saatweek/Intel-Image-Classification/blob/main/Intel_Image_Classification_(using_Transfer_Learning).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#The dataset is stored in my Google Drive, so we'll first have to connect this notebook 
#to the Google drive to access the files in the drive from this notebook 

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#importing dependencies

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import plotly.graph_objects as go
import tensorflow_hub as hub

In [ ]:
train_dir = '/content/drive/My Drive/Colab Notebooks/Personal Projects/Intel Image Classification/seg_train'
validation_dir = '/content/drive/My Drive/Colab Notebooks/Personal Projects/Intel Image Classification/seg_test'

train_datagen = ImageDataGenerator(rescale = 1./255, rotation_range=0.3, width_shift_range=0.3,
    height_shift_range=0.3,shear_range=0.3, zoom_range=0.3, horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale = 1./255, rotation_range=0.3, width_shift_range=0.3,
    height_shift_range=0.3,shear_range=0.3, zoom_range=0.3, horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(train_dir, target_size = (150, 150), batch_size = 128, class_mode='sparse')

validation_generator = validation_datagen.flow_from_directory(validation_dir, target_size = (150, 150), batch_size = 128, class_mode='sparse')

Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.


In [ ]:
url = 'https://tfhub.dev/google/imagenet/inception_v3/feature_vector/4'

base_model = hub.KerasLayer(url, input_shape = (150, 150, 3))

model = tf.keras.Sequential([base_model, 
                             tf.keras.layers.Dense(128, activation='relu'),
                             tf.keras.layers.Dense(64, activation='relu'),
                             tf.keras.layers.Dense(6, activation='softmax')])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_1 (KerasLayer)   (None, 2048)              21802784  
_________________________________________________________________
dense (Dense)                (None, 128)               262272    
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 390       
Total params: 22,073,702
Trainable params: 270,918
Non-trainable params: 21,802,784
_________________________________________________________________


In [ ]:
model.compile(loss = 'sparse_categorical_crossentropy', optimizer='adam', metrics = 'acc')

In [ ]:
history = model.fit(train_generator, steps_per_epoch=20, epochs=10, validation_data = validation_generator)

Epoch 1/10
20/20 [==============================] - 3995s 200s/step - loss: 1.0758 - acc: 0.6113 - val_loss: 0.5951 - val_acc: 0.7977
Epoch 2/10
20/20 [==============================] - 1458s 73s/step - loss: 0.5515 - acc: 0.8121 - val_loss: 0.5326 - val_acc: 0.8073
Epoch 3/10
20/20 [==============================] - 1210s 60s/step - loss: 0.5155 - acc: 0.8152 - val_loss: 0.5218 - val_acc: 0.8203
Epoch 4/10
20/20 [==============================] - 959s 48s/step - loss: 0.5215 - acc: 0.8109 - val_loss: 0.5622 - val_acc: 0.8027
Epoch 5/10
20/20 [==============================] - 769s 38s/step - loss: 0.4841 - acc: 0.8211 - val_loss: 0.4492 - val_acc: 0.8400
Epoch 6/10
20/20 [==============================] - 609s 30s/step - loss: 0.4781 - acc: 0.8316 - val_loss: 0.4367 - val_acc: 0.8347
Epoch 7/10
20/20 [==============================] - 524s 26s/step - loss: 0.4408 - acc: 0.8409 - val_loss: 0.4394 - val_acc: 0.8490
Epoch 8/10
20/20 [==============================] - 454s 23s/step - loss

In [ ]:
#PLotting the graph

fig = go.Figure()
x = np.linspace(1,10, 10)
y1 = history.history['acc']
y2 = history.history['val_acc']
fig.add_trace(go.Scatter(x = x, y = y1, name = 'training accuracy')),
fig.add_trace(go.Scatter(x = x, y = y2, name = 'validation accuracy')),
fig.update_layout(xaxis_title = 'Epochs', yaxis_title = 'accuracy', title = 'Accuracy of Model')
fig.show()

In [ ]:
model.save_weights('/content/drive/My Drive/Colab Notebooks/Personal Projects/Intel Image Classification/')